<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Text Term Frequency Analysis (Python-SQL)</b>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>
This demo will analyse the text in rows of the table to find the TF-IDF or Term Frequency-Inverse Document Frequency is an indicator of a term's importance in a specific document based on the entire corpus of documents.    
This is a demonstration of Vantage capabilities for functional demos e.g.
    <li style = 'font-size:16px;font-family:Arial'> NGramSplitter Function - tokenizes (splits) an input stream of text and outputs n multigrams (called n-grams) based on the specified Reset, Punctuation, and Delimiter syntax elements.</li>
</p>
<p style = 'font-size:16px;font-family:Arial'> This notebook demonstrate how the function is used in Python kernel, there is a similar notebook which shows the same features in sql kernel. </p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Steps</b></p>
<p style = 'font-size:16px;font-family:Arial'>
    <li style = 'font-size:16px;font-family:Arial'> Connect to Vantage and read the dataset. </li>
    <li style = 'font-size:16px;font-family:Arial'> Use NGramSplitter SQL to create a table of grams of n-size. </li>
    <li style = 'font-size:16px;font-family:Arial'> Express SQL to calculate TF-IDF and store the output in a table. </li>
    <li style = 'font-size:16px;font-family:Arial'> Retrieve the data as a local dataframe. </li>
    <li style = 'font-size:16px;font-family:Arial'> Basic visualization to show top 30 important terms. </li>
</p>

<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>1. Import python packages, connect to Vantage and explore the dataset</b></p>

In [ ]:
import getpass
import warnings

import pandas as pd
import numpy as np

from teradataml.dataframe.dataframe import DataFrame
from teradataml.analytics.sqle import NGramSplitter
from teradataml.dataframe.dataframe import in_schema
from teradataml.context.context import create_context, remove_context, get_context
from teradataml.dataframe.copy_to import copy_to_sql
from teradataml.options.display import display

from teradatasqlalchemy.types import *

import matplotlib.pyplot as plt

%matplotlib inline

warnings.filterwarnings('ignore')
display.max_rows = 5

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell.</p>

In [ ]:
%run -i ../startup.ipynb

<p style = 'font-size:16px;font-family:Arial'>Below command will make a connection to the Vantage environment.

In [ ]:
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
eng.execute('''SET query_band='DEMO=Text_Term_Frequency_PY_SQL.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys.</p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. In this demo since we are using Temporal table we will be creating databases and tables in local storage and use them in the notebook. Please execute the procedure in the next cell.</p>

In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_Retail_cloud');"
 # takes about 25 seconds, estimated space: 0 MB
#%run -i ../run_procedure.py "call get_data('DEMO_Retail_local');" 
# takes about 50 seconds, estimated space: 23 MB

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<p style = 'font-size:16px;font-family:Arial'>Get the data from Vantage in the DataFrame.</p>   

In [ ]:
tdf_reviews = DataFrame('"DEMO_Retail"."Web_Comment"')

<p style = 'font-size:16px;font-family:Arial'>Let us see how the data in the table looks like. We have taken one comment.</p>

In [ ]:
qry = '''
SELECT *
FROM "DEMO_Retail"."Web_Comment" where comment_id = 30
'''
tdf_res = DataFrame.from_query(qry)
tdf_res

<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>2. Use the NGram Splitter SQL Function</b></p>
<p style = 'font-size:16px;font-family:Arial'>NGram function will split the corpus of documents into "terms" (grams) of selected size.  Specifically, this example will create a table called "tbl_grams" that is the result of splitting each "document" (review) into two-word chunks (grams).  Each row in this table includes;
<ol style = 'font-size:16px;font-family:Arial'>
    <li>The two-word chunk (ngram).</li>
    <li>The source review id (row_id).</li>
     <li>Chunk length (n).</li>
     <li>The count of this chunk in the review (frequency).</li>
     <li>The count of this chunk in all the reviews (totalcnt)</li>
</ol>
<p style = 'font-size:16px;font-family:Arial'>
The splitting algorithm can be controlled with delimeters, punctuation indicators, etc.</p>

In [ ]:
qry = 'DROP TABLE tbl_grams;'

try:
    eng.execute(qry)
except Exception as e:
    if str(e.args).find('3807') >= 1:
        pass
    else:
        raise

#how many grams should we split the docs into?
grams = 2

#Create ngram table
qry = f'''
CREATE MULTISET TABLE tbl_grams AS (
    SELECT * FROM NGramSplitter ( 
        ON ( SELECT * FROM "DEMO_Retail"."Web_Comment" )   
        USING 
            TextColumn('comment_text') 
            Accumulate('comment_id') 
            Grams('{grams}') 
            OverLapping('TRUE') 
            ConvertToLowerCase('TRUE') 
            Delimiter(' ') 
            Punctuation('[`~#^&*()-]') 
            OutputTotalGramCount('TRUE') 
            NGramColName('ngram') 
            GramLengthColName('n') 
            FrequencyColName('frequency') 
            TotalCountColName('totalcnt') 
    ) as ngram_out
    )
WITH DATA
PRIMARY INDEX (comment_id);
'''

eng.execute(qry)

tdf_grams = DataFrame('tbl_grams')

tdf_grams

<p style = 'font-size:16px;font-family:Arial'>In the table created above we can see the NGram function applied to the web comment column. We can see the frequency and the total number of times the ngram appear in the column.</p>

<p style = 'font-size:16px;font-family:Arial'>Let us check how the comment id we saw earlier looks after converting to ngrams</p>

In [ ]:
qry = '''
SELECT *
FROM tbl_grams where comment_id = 30 
'''
tdf_res = DataFrame.from_query(qry)
tdf_res

<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>3. Create the TF-IDF Table</b></p>
<p style = 'font-size:16px;font-family:Arial'><b>TF-IDF</b> or <b>Term Frequency-Inverse Document Frequency</b> is an indicator of a term's <b>importance</b> in a specific document based on the entire corpus of documents.  This value is calculated by taking the Product of:
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Term Frequency = (Number of Terms in the Document)/(Number of Terms in the Corpus)</li>
    <li>Inverse Document Frequency = Natural Log((Total Number of Documents)/(Number of Documents with the Term))</li>
 </ul>   
<p style = 'font-size:16px;font-family:Arial'>
This can be accomplished in SQL using the results table created using NGgram Splitter function in above step:</p>

In [ ]:
# Drop the table if it already exists
qry = 'DROP TABLE tbl_tf_idf;'
try:
    eng.execute(qry)
except Exception as e:
    if str(e.args).find('3807') >= 1:
        pass
    else:
        raise

# Create the table
qry = '''
CREATE MULTISET TABLE tbl_tf_idf AS (
    SELECT gr.comment_id AS comment_id,
    gr."ngram" AS term,
    CAST(CAST(gr.frequency AS FLOAT) / CAST(gr.totalcnt AS FLOAT) AS DECIMAL(10,6)) AS tf,
    CAST(LN(sel_docs.tot_docs / sel_docs.tot_term) AS DECIMAL(10,6)) AS idf,
    CAST(idf * tf AS DECIMAL(10,6)) AS tf_idf
    FROM tbl_grams AS gr
    -- get the number of docs where each term exists
    LEFT JOIN (
        SELECT "ngram", tot_term , tot_docs FROM (
            (SELECT "ngram", COUNT(*) AS tot_term
            FROM tbl_grams
            GROUP BY "ngram") terms
            -- get the total doc count and join it to the table
            CROSS JOIN (SELECT COUNT(DISTINCT comment_id) AS tot_docs FROM tbl_grams ) AS sum_docs
        )
    ) sel_docs
    ON gr."ngram" = sel_docs."ngram"
    WHERE tf_idf > .5
)
WITH DATA
PRIMARY INDEX (comment_id);
'''

# Execute the query
eng.execute(qry)

# Get the data from the created table
tdf_tf_idf = DataFrame('tbl_tf_idf')
tdf_tf_idf

<p style = 'font-size:16px;font-family:Arial'>Let us check the term frequency and inverse document frequency calculated for the comment we saw before</p>

In [ ]:
qry = '''
SELECT *
FROM tbl_tf_idf where comment_id = 30
'''
tdf_res = DataFrame.from_query(qry)
tdf_res

<p style = 'font-size:16px;font-family:Arial'>Now, let us check the mostly used terms in our data</p>

In [ ]:
df_tf_idf = tdf_tf_idf.to_pandas(all_rows = True)
df_tf_idf['tf_idf'] = df_tf_idf['tf_idf'].astype(float)

In [ ]:
df_tf_idf.sort_values(by = 'tf_idf', ascending = False)

<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>4. Visualize the Results</b></p>
<p style = 'font-size:16px;font-family:Arial'>
Let's use Pandas and Matplotlib to do visualizations of the data:</p>


In [ ]:
df1 = df_tf_idf.sort_values(by = 'tf_idf', ascending = False).head(30)

In [ ]:
#plot it:
df1.sort_values(by = 'tf_idf', ascending = True).set_index('term')[['tf_idf']].plot(kind = 'barh', legend = True, figsize = (12, 9));

<p style = 'font-size:16px;font-family:Arial'>
In this plot we are plotting the top 30 terms which are used in the reviews. </p>

<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>5.  Clean up 
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>Worktables </b></p>

In [ ]:
eng.execute('DROP TABLE DEMO_USER.tbl_tf_idf;')

In [ ]:
eng.execute('DROP TABLE DEMO_USER.tbl_grams;')

In [ ]:
remove_context()

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>Database and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_Retail');" 
#Takes 5 seconds

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Reference Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradata Python Package User Guide: <a href = 'https://docs.teradata.com/reader/eteIDCTX4O4IMvazRMypxQ/uDjppX7PJInABCckgu~KFg'>https://docs.teradata.com/reader/eteIDCTX4O4IMvazRMypxQ/uDjppX7PJInABCckgu~KFg</a></li>
    <li>Teradataml Python Reference: <a href = 'https://docs.teradata.com/reader/GsM0pYRZl5Plqjdf9ixmdA/MzdO1q_t80M47qY5lyImOA'>https://docs.teradata.com/reader/GsM0pYRZl5Plqjdf9ixmdA/MzdO1q_t80M47qY5lyImOA</a></li>
    <li>Teradata NGramSplitter Function Reference: <a href = 'https://docs.teradata.com/r/Teradata-VantageTM-Analytics-Database-Analytic-Functions-17.20/Text-Analytic-Functions/NGramSplitter'>https://docs.teradata.com/r/Teradata-VantageTM-Analytics-Database-Analytic-Functions-17.20/Text-Analytic-Functions/NGramSplitter</a></li>
  
</ul>


<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2023 Teradata. All Rights Reserved</footer>